# **Data Loading and Text Processing for BERT**

In [5]:
# !pip install numpy==1.26.0
# !pip install torch==2.2.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
# !pip install torchtext==0.17.2
# !pip install torchdata==0.7.1
!pip install portalocker==2.8.2
# !pip install pandas==2.2.1
# !pip install transformers==4.35.2

In [6]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchtext.vocab import build_vocab_from_iterator
from torchtext.vocab import Vocab
from torch import Tensor
from torch.nn import Transformer
from torchtext.data.utils import get_tokenizer
from torch.nn.utils.rnn import pad_sequence
from itertools import chain
from itertools import islice
from torchtext.datasets import IMDB
from copy import deepcopy
import random
import csv
import json
from tqdm import tqdm
import pandas as pd

# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

## **1. Tokenization and Vocabulary Building**

In [7]:
tokenizer = get_tokenizer("basic_english")

def yield_tokens(data_iter):
    for label, data_sample in data_iter:
        yield tokenizer(data_sample)

PAD_IDX,CLS_IDX, SEP_IDX,  MASK_IDX,UNK_IDX= 0, 1, 2, 3, 4
special_symbols = ['[PAD]','[CLS]', '[SEP]','[MASK]','[UNK]']

In [8]:
train_iter, test_iter = IMDB(split=('train', 'test'))
all_data_iter = chain(train_iter, test_iter)

vocab=build_vocab_from_iterator(yield_tokens(all_data_iter),specials=special_symbols,special_first=True)
vocab.set_default_index(UNK_IDX)
VOCAB_SIZE=len(vocab)
print(VOCAB_SIZE)

147161


In [9]:
text_to_index=lambda text: [vocab(token) for token in tokenizer(text)]
index_to_en = lambda seq_en: " ".join([vocab.get_itos()[index] for index in seq_en])

## **2. Text Masking and Data Preparation for BERT**

### Text Masking

In [10]:
def bernoulli_true_false(p):
    bernoulli_dist = torch.distributions.Bernoulli(torch.tensor([p]))  # probability p
    return bernoulli_dist.sample().item() == 1  # convert 1 to True and 0 to False

In [11]:
def Masking(token):
    mask = bernoulli_true_false(0.2)  # Decide whether to mask this token (20% chance)
    # If mask is False, immediately return with '[PAD]' label
    if not mask:
        return token, '[PAD]'

    # If mask is True, proceed with further operations
    # Randomly decide on an operation (50% chance each)
    random_opp = bernoulli_true_false(0.5)
    random_swich = bernoulli_true_false(0.5)

    # Case 1: If mask, random_opp, and random_swich are True (Masked)
    if mask and random_opp and random_swich:
        # Replace the token with '[MASK]' and set label to a random token
        mask_label = index_to_en(torch.randint(0, VOCAB_SIZE, (1,)))
        token_ = '[MASK]'

    # Case 2: If mask and random_opp are True, but random_swich is False (Unchanged)
    elif mask and random_opp and not random_swich:
        # Leave the token unchanged and set label to the same token
        token_ = token
        mask_label = token

    # Case 3: If mask is True, but random_opp is False (Noise)
    else:
        # Replace the token with '[MASK]' and set label to the original token
        token_ = '[MASK]'
        mask_label = token

    return token_, mask_label

### Data Preparation for MLM

In [12]:
def prepare_for_mlm(tokens, include_raw_tokens=False):
    """
    Prepares tokenized text for BERT's Masked Language Model (MLM) training.
    """
    bert_input = []  # List to store sentences processed for BERT's MLM
    bert_label = []  # List to store labels for each token (mask, random, or unchanged)
    raw_tokens_list = []  # List to store raw tokens if needed
    current_bert_input = []
    current_bert_label = []
    current_raw_tokens = []

    for token in tokens:
        # Apply BERT's MLM masking strategy to the token
        masked_token, mask_label = Masking(token)

        # Append the processed token and its label to the current sentence and label list
        current_bert_input.append(masked_token)
        current_bert_label.append(mask_label)

        # If raw tokens are to be included, append the original token to the current raw tokens list
        if include_raw_tokens:
            current_raw_tokens.append(token)

        # Check if the token is a sentence delimiter (., ?, !)
        if token in ['.', '?', '!']:
            # If current sentence has more than two tokens, consider it a valid sentence
            if len(current_bert_input) > 2:
                bert_input.append(current_bert_input)
                bert_label.append(current_bert_label)
                # If including raw tokens, add the current list of raw tokens to the raw tokens list
                if include_raw_tokens:
                    raw_tokens_list.append(current_raw_tokens)

                # Reset the lists for the next sentence
                current_bert_input = []
                current_bert_label = []
                current_raw_tokens = []
            else:
                # If the current sentence is too short, discard it and reset lists
                current_bert_input = []
                current_bert_label = []
                current_raw_tokens = []

    # Add any remaining tokens as a sentence if there are any
    if current_bert_input:
        bert_input.append(current_bert_input)
        bert_label.append(current_bert_label)
        if include_raw_tokens:
            raw_tokens_list.append(current_raw_tokens)

    # Return the prepared lists for BERT's MLM training
    return (bert_input, bert_label, raw_tokens_list) if include_raw_tokens else (bert_input, bert_label)

### Data Preparation for NSP

In [13]:
def process_for_nsp(input_sentences, input_masked_labels):
    """
    Prepares data for Next Sentence Prediction (NSP) task in BERT training.

    Args:
    input_sentences (list): List of tokenized sentences.
    input_masked_labels (list): Corresponding list of masked labels for the sentences.

    Returns:
    bert_input (list): List of sentence pairs for BERT input.
    bert_label (list): List of masked labels for the sentence pairs.
    is_next (list): Binary label list where 1 indicates 'next sentence' and 0 indicates 'not next sentence'.
    """
    if len(input_sentences) < 2:
       raise ValueError("must have two same number of items.")


    # Verify that both input lists are of the same length and have a sufficient number of sentences
    if len(input_sentences) != len(input_masked_labels):
        raise ValueError("Both lists must have the same number of items.")

    bert_input = []
    bert_label = []
    is_next = []

    available_indices = list(range(len(input_sentences)))

    while len(available_indices) >= 2:
        if random.random() < 0.5:
            # Choose two consecutive sentences to simulate the 'next sentence' scenario
            index = random.choice(available_indices[:-1])  # Exclude the last index
            # append list and add  '[CLS]' and  '[SEP]' tokens
            bert_input.append([['[CLS]']+input_sentences[index]+ ['[SEP]'],input_sentences[index + 1]+ ['[SEP]']])
            bert_label.append([['[PAD]']+input_masked_labels[index]+['[PAD]'], input_masked_labels[index + 1]+ ['[PAD]']])
            is_next.append(1)  # Label 1 indicates these sentences are consecutive

            # Remove the used indices
            available_indices.remove(index)
            if index + 1 in available_indices:
                available_indices.remove(index + 1)
        else:
            # Choose two random distinct sentences to simulate the 'not next sentence' scenario
            indices = random.sample(available_indices, 2)
            bert_input.append([['[CLS]']+input_sentences[indices[0]]+['[SEP]'],input_sentences[indices[1]]+ ['[SEP]']])
            bert_label.append([['[PAD]']+input_masked_labels[indices[0]]+['[PAD]'], input_masked_labels[indices[1]]+['[PAD]']])
            is_next.append(0)  # Label 0 indicates these sentences are not consecutive

            # Remove the used indices
            available_indices.remove(indices[0])
            available_indices.remove(indices[1])



    return bert_input, bert_label, is_next

### Finalizing BERT Inputs

In [14]:
def prepare_bert_final_inputs(bert_inputs, bert_labels, is_nexts,to_tenor=True):
    """
    Prepare the final input lists for BERT training.
    """
    def zero_pad_list_pair(pair_, pad='[PAD]'):
        pair=deepcopy(pair_)
        max_len = max(len(pair[0]), len(pair[1]))
        #append [PAD] to each sentence in the pair till the maximum length reaches
        pair[0].extend([pad] * (max_len - len(pair[0])))
        pair[1].extend([pad] * (max_len - len(pair[1])))
        return pair[0], pair[1]

    #flatten the tensor
    flatten = lambda l: [item for sublist in l for item in sublist]
    #transform tokens to vocab indices
    tokens_to_index=lambda tokens: [vocab[token] for token in tokens]

    bert_inputs_final, bert_labels_final, segment_labels_final, is_nexts_final = [], [], [], []

    for bert_input, bert_label,is_next in zip(bert_inputs, bert_labels,is_nexts):
        # Create segment labels for each pair of sentences
        segment_label = [[1] * len(bert_input[0]), [2] * len(bert_input[1])]

        # Zero-pad the bert_input and bert_label and segment_label
        bert_input_padded = zero_pad_list_pair(bert_input)
        bert_label_padded = zero_pad_list_pair(bert_label)
        segment_label_padded = zero_pad_list_pair(segment_label,pad=0)

        #convert to tensors
        if to_tenor:

            # Flatten the padded inputs and labels, transform tokens to their corresponding vocab indices, and convert them to tensors
            bert_inputs_final.append(torch.tensor(tokens_to_index(flatten(bert_input_padded)),dtype=torch.int64))
            #bert_labels_final.append(torch.tensor(tokens_to_index(flatten(bert_label_padded)),dtype=torch.int64))
            bert_labels_final.append(torch.tensor(tokens_to_index(flatten(bert_label_padded)),dtype=torch.int64))
            segment_labels_final.append(torch.tensor(flatten(segment_label_padded),dtype=torch.int64))
            is_nexts_final.append(is_next)

        else:
          # Flatten the padded inputs and labels
            bert_inputs_final.append(flatten(bert_input_padded))
            bert_labels_final.append(flatten(bert_label_padded))
            segment_labels_final.append(flatten(segment_label_padded))
            is_nexts_final.append(is_next)

    return bert_inputs_final, bert_labels_final, segment_labels_final, is_nexts_final


## **3. Preparing the Dataset**

In [15]:
csv_file_path ='train_bert_data_new.csv'
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    csv_writer = csv.writer(file)
    csv_writer.writerow(['Original Text', 'BERT Input', 'BERT Label', 'Segment Label', 'Is Next'])

    # Wrap train_iter with tqdm for a progress bar
    for n, (_, sample) in enumerate(tqdm(train_iter, desc="Processing samples")):
        # Tokenize the sample input
        tokens = tokenizer(sample)
        # Create MLM inputs and labels
        bert_input, bert_label = prepare_for_mlm(tokens, include_raw_tokens=False)
        if len(bert_input) < 2:
            continue
        # Create NSP pairs, token labels, and is_next label
        bert_inputs, bert_labels, is_nexts = process_for_nsp(bert_input, bert_label)
        # add zero-paddings, map tokens to vocab indices and create segment labels
        bert_inputs, bert_labels, segment_labels, is_nexts = prepare_bert_final_inputs(bert_inputs, bert_labels, is_nexts)
        # convert tensors to lists, convert lists to JSON-formatted strings
        for bert_input, bert_label, segment_label, is_next in zip(bert_inputs, bert_labels, segment_labels, is_nexts):
            bert_input_str = json.dumps(bert_input.tolist())
            bert_label_str = json.dumps(bert_label.tolist())
            segment_label_str = ','.join(map(str, segment_label.tolist()))
            # Write the data to a CSV file row-by-row
            csv_writer.writerow([sample, bert_input_str, bert_label_str, segment_label_str, is_next])

Processing samples: 25000it [2:09:07,  3.23it/s]


In [17]:
from torch.utils.data import Dataset

class BertPretrainDataset(Dataset):
    def __init__(self, csv_path):
        self.data = pd.read_csv(csv_path)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        bert_input = torch.tensor(json.loads(self.data.iloc[idx]['BERT Input']), dtype=torch.long)
        bert_label = torch.tensor(json.loads(self.data.iloc[idx]['BERT Label']), dtype=torch.long)
        segment_label = torch.tensor([int(x) for x in self.data.iloc[idx]['Segment Label'].split(',')], dtype=torch.long)
        is_next = torch.tensor(int(self.data.iloc[idx]['Is Next']), dtype=torch.long)

        return {'input_ids': bert_input,
                'labels': bert_label,
                'token_type_ids': segment_label,
                'next_sentence_label': is_next}

In [23]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    input_ids = [item['input_ids'] for item in batch]
    labels = [item['labels'] for item in batch]
    token_type_ids = [item['token_type_ids'] for item in batch]
    next_sentence_label = torch.stack([item['next_sentence_label'] for item in batch])

    # Pad sequences to same length within the batch
    input_ids = pad_sequence(input_ids, batch_first=True, padding_value=0)
    labels = pad_sequence(labels, batch_first=True, padding_value=-100)  # -100 is ignored in loss
    token_type_ids = pad_sequence(token_type_ids, batch_first=True, padding_value=0)

    return {
        'input_ids': input_ids,
        'labels': labels,
        'token_type_ids': token_type_ids,
        'next_sentence_label': next_sentence_label
    }


In [24]:
dataset = BertPretrainDataset('train_bert_data_new.csv')
train_loader = DataLoader(dataset, batch_size=32, shuffle=True, drop_last=True, collate_fn=collate_fn)

In [25]:
first_batch = next(iter(train_loader))

for i in range(2):
    print(f"\nBatch Sample {i}:")
    for key in first_batch:
        print(f"{key}: {first_batch[key][i]}")


Batch Sample 0:
input_ids: tensor([  1,  23,   3, 123,  10,   3,   3,   2,  98,  12,  30,   3,   2,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0